In [3]:
import json
from pathlib import Path

dataset_root = "D:\David\Škola\RP\dataset"
dataset_filename = "train_no_dup.json"

def process_dataset():
    invalid_categories = [77, 76, 78, 113, 115, 116, 118, 120, 122, 123, 124, 126, 127, 129, 130, 132, 135, 136, 139, 140, 141, 143, 144, 4241, 4242, 147, 4244, 150, 4247, 4248, 153, 156, 154, 155, 157, 4254, 159, 160, 4257, 162, 163, 164, 166, 167, 168, 169, 170, 4267, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 231, 311, 313, 314, 316, 317, 321, 4432, 4433, 4437, 4439, 4440, 4441, 4442, 4443, 4445, 4446, 4448, 4449, 4450, 4451, 4478, 4480, 4481, 4482, 4483, 4484, 4485, 4486, 4487, 4488, 4489, 4490, 4492, 4493, 4499, 4500, 4501, 4502, 4503, 4504, 4505, 4506, 4507, 4508, 4509, 4510, 4511, 4512, 4513, 4512, 4438, 4240, 146, 148, 4246, 151, 152, 949, 161, 4258, 171, 4269, 4276, 3336, 1967, 4431, 5535, 4436, 4430,
                93, 94, 95, 96, 97, 98, 99, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 4292, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 211, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 319, 320, 333, 334, 335, 338, 339, 340, 196, 336, 337]
    total_disposed = 0
    outfits_disposed = 0
    total_items = 0
    total_outfits = 0
    categories = []
    with open(Path(dataset_root, dataset_filename)) as json_file:
        raw_json = json.load(json_file)
        print("Loaded " + str(len(raw_json)) + " items", flush=True)
        examples = []
        for outfit in raw_json:
            images = []
            
            for item in outfit["items"]:
                if item["categoryid"] in invalid_categories:
                    total_disposed = total_disposed + 1
                    continue
                total_items = total_items + 1
                categories.append(item["categoryid"])
                images.append(1)
            if len(images) < 3:
                outfits_disposed = outfits_disposed + 1
                continue
            
            total_outfits = total_outfits + 1


    print("Total " + str(total_items) + " products")
    print("Total " + str(total_outfits) + " outfits")
    return examples

process_dataset()

Loaded 17316 items
Total 93963 products
Total 16777 outfits


[]